<a href="https://colab.research.google.com/github/Nivratti/HeadPoseEstimation-WHENet/blob/master/HeadPoseEstimation_WHENet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color="orange" size="+3">1. setup : </font>

In [1]:
!git clone https://github.com/Nivratti/HeadPoseEstimation-WHENet.git

Cloning into 'HeadPoseEstimation-WHENet'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 268 (delta 67), reused 246 (delta 56), pack-reused 0
Receiving objects: 100% (268/268), 24.04 MiB | 15.08 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [2]:
## Install retinaface -- face detection library 
# --no-dependencies : Ignore package dependencies -- otherwise it will install some specific version libraries -- it will take time and sometimes we need to restart kernel to laod new packages
!pip install --no-dependencies retinaface 
!pip install --no-dependencies utilpack ## it installing new pillow version -- and we need to restart -- so we are not installing any dependancies
!pip install slack_sdk
!pip install pdfrw
!pip install pycryptodome
!pip install PyMySQL

     |████████████████████████████████| 5.9MB 12.9MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 235kB 13.2MB/s 
ERROR: utilpack 2.0.0 requires gunicorn==20.1.0, which is not installed.
ERROR: utilpack 2.0.0 requires pdfrw==0.4, which is not installed.
ERROR: utilpack 2.0.0 requires pycryptodome==3.10.1, which is not installed.
ERROR: utilpack 2.0.0 requires PyMySQL==1.0.2, which is not installed.
ERROR: utilpack 2.0.0 has requirement Flask==2.0.1, but you'll have flask 1.1.4 which is incompatible.
ERROR: utilpack 2.0.0 has requirement matplotlib==3.3.4, but you'll have matplotlib 3.2.2 which is incompatible.
ERROR: utilpack 2.0.0 has requirement opencv-python==4.5.2.54, but you'll have opencv-python 4.1.2.30 which is incompatible.
ERROR: utilpack 2.0.0 has requirement Pillow==8.3.1, but you'll have pillow 7.1.2 which is incompatible.
ERROR: utilpack 2.0.0 has requirement requests==2.25.1, but you'll have requests 2.23.0 which is inc

In [3]:
## Auto time plugin
!pip install ipython-autotime

%load_ext autotime

time: 151 µs (started: 2021-07-17 14:17:23 +00:00)


In [4]:
!pip install nb_utils

time: 2.74 s (started: 2021-07-17 14:17:23 +00:00)


In [5]:
!pip install loguru

     |████████████████████████████████| 61kB 6.8MB/s 
time: 2.83 s (started: 2021-07-17 14:17:26 +00:00)


In [6]:
!pip install efficientnet

     |████████████████████████████████| 51kB 6.7MB/s 
time: 2.84 s (started: 2021-07-17 14:17:29 +00:00)


## <font color="orange" size="+3">2. Logger configuration: </font>


In [7]:
## Loge level, formatter
import sys
from loguru import logger

logger.remove() # remove previously added handlar - -to make a fresh start.

# enqueue=True : Asynchronous, Thread-safe, Multiprocess-safe
# backtrace=True, diagnose=True : Fully descriptive exceptions - Loguru helps you identify problems by allowing the entire stack trace to be displayed, including values of variables
logger.add(sys.stdout, colorize=True, format="<green>{time:HH:mm:ss}</green> | {level} | {name} | <level>{message}</level>", level="INFO", enqueue=True, backtrace=True, diagnose=True)

1

time: 28.2 ms (started: 2021-07-17 14:17:46 +00:00)


In [8]:
logger.info("Hi")

time: 1.61 ms (started: 2021-07-17 14:17:48 +00:00)
14:17:48 | INFO | __main__ | Hi


## <font color="orange" size="+3">3. Write Retinaface wrapper and init detector : </font>

Custom wrapper for reducing image size for speed improvement

In [9]:
#@title Define FaceDetection_RetinaFaceTF2
import cv2 
import imutils
from retinaface import RetinaFace

class FaceDetection_RetinaFaceTF2:
    def __init__(self, quality="normal"):
        self.face_detector = RetinaFace(quality=quality)

    def read(self, image_path):
        return self.face_detector.read(image_path)

    def detect_highlight_face(self, rgb_image, max_side_len=640, max_faces=1, return_boxformat="xywh"):
        ## Resize image
        scale_factor = None

        height, width, c = rgb_image.shape
        if width > max_side_len or height > max_side_len:
            if height > width:
                scale_factor = max_side_len / height

                resized = imutils.resize(rgb_image, height=max_side_len)
            else:
                scale_factor = max_side_len / width
                resized = imutils.resize(rgb_image, width=max_side_len)

            logger.info(f"scale_factor: {scale_factor}")
            # tn_image = resize((int(width * scale_factor), int(height * scale_factor)))

        if scale_factor:
            faces = self.face_detector.predict(resized)

            ## Rescaled face boxes -- restore
            if faces:
              for face in faces:
                  face["x1"] = int(face["x1"] / scale_factor)
                  face["y1"] = int(face["y1"] / scale_factor)
                  face["x2"] = int(face["x2"] / scale_factor)
                  face["y2"] = int(face["y2"] / scale_factor)
        else:
            faces = self.face_detector.predict(rgb_image)
    
        # print(faces)

        # faces is list of face dictionary
        # each face dictionary contains x1 y1 x2 y2 left_eye right_eye nose left_lip right_lip
        # faces=[{"x1":20,"y1":32, ... }, ...]

        if faces:
            result_img = self.face_detector.draw(rgb_image, faces)
        else:
            result_img = None
        # # save ([...,::-1] : rgb -> bgr )
        # cv2.imwrite("result_img.jpg",result_img[...,::-1])

        return (faces, result_img)

time: 3.04 s (started: 2021-07-17 14:18:24 +00:00)


In [10]:
face_detector = FaceDetection_RetinaFaceTF2()
logger.info("Success! Initialized face detector..")

model[normal quality] init ..
model success !
time: 19.6 s (started: 2021-07-17 14:18:27 +00:00)
14:18:46 | INFO | __main__ | Success! Initialized face detector..


## <h1><font color="orange" size="+3">4. Test detector : </font></h1>

In [41]:
## RetinaFaceTF2 -- takes less than 100 ms on GPU

# same with cv2.imread,cv2.cvtColor 
rgb_image = face_detector.read("/content/HeadPoseEstimation-WHENet/Sample/maruti.jpg")

faces, result_img = face_detector.detect_highlight_face(rgb_image, max_side_len=640)

logger.info(f"Total faces: {len(faces)}")
logger.info(f"faces: {faces}")

## RGB to BGR
cv2.imwrite("result_img.jpg", result_img[...,::-1])

14:59:12 | INFO | __main__ | scale_factor: 0.16740779492545121
14:59:13 | INFO | __main__ | Total faces: 1
14:59:13 | INFO | __main__ | faces: [{'x1': 1756, 'y1': 842, 'x2': 2234, 'y2': 1469, 'left_eye': (317, 180), 'right_eye': (350, 189), 'nose': (327, 200), 'left_lip': (310, 215), 'right_lip': (338, 222)}]


True

time: 614 ms (started: 2021-07-17 14:59:12 +00:00)


In [42]:
## Modified facebox format
face = [faces[0]["x1"], faces[0]["y1"], faces[0]["x2"], faces[0]["y2"]]
logger.info(f"face: {face} \n")

14:59:19 | INFO | __main__ | face: [1756, 842, 2234, 1469] 

time: 4.8 ms (started: 2021-07-17 14:59:19 +00:00)


## <font color="orange" size="+3">2. Logger configuration: </font>


In [43]:
%cd "/content/HeadPoseEstimation-WHENet"

/content/HeadPoseEstimation-WHENet
time: 1.69 ms (started: 2021-07-17 14:59:34 +00:00)


In [44]:
import os
import cv2
import numpy as np
from PIL import Image
from utils import draw_axis


def process_detection( model, img_bgr, bbox, is_draw_axis=True, display_all_euler_angle="full"):

    img = img_bgr
    # y_min, x_min, y_max, x_max = bbox # yolo v3 face detector
    x_min, y_min, x_max, y_max = bbox # retinaface tf2

    # enlarge the bbox to include more background margin
    y_min = max(0, y_min - abs(y_min - y_max) / 10)
    y_max = min(img.shape[0], y_max + abs(y_min - y_max) / 10)
    x_min = max(0, x_min - abs(x_min - x_max) / 5)
    x_max = min(img.shape[1], x_max + abs(x_min - x_max) / 5)
    x_max = min(x_max, img.shape[1])

    img_cropped = img[int(y_min):int(y_max), int(x_min):int(x_max)]
    # cv2.imwrite("img_cropped.jpg", img_cropped)
    
    img_rgb = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2RGB)
    img_rgb = cv2.resize(img_rgb, (224, 224))
    img_rgb = np.expand_dims(img_rgb, axis=0)

    cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0,0,0), 2)
    yaw, pitch, roll = model.get_angle(img_rgb)
    yaw, pitch, roll = np.squeeze([yaw, pitch, roll])

    if is_draw_axis:
        draw_axis(img, yaw, pitch, roll, tdx=(x_min+x_max)/2, tdy=(y_min+y_max)/2, size = abs(x_max-x_min)//2 )

    if display_all_euler_angle == 'full':
        cv2.putText(img, "yaw: {}".format(np.round(yaw)), (int(x_min), int(y_min)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 255, 0), 1)
        cv2.putText(img, "pitch: {}".format(np.round(pitch)), (int(x_min), int(y_min) - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 255, 0), 1)
        cv2.putText(img, "roll: {}".format(np.round(roll)), (int(x_min), int(y_min)-30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 255, 0), 1)
    return (img, yaw, pitch, roll)


time: 22.5 ms (started: 2021-07-17 14:59:35 +00:00)


In [15]:
from whenet import WHENet

time: 1.12 ms (started: 2021-07-17 14:21:18 +00:00)


In [16]:
import os
import argparse

args = argparse.Namespace(
    snapshot="WHENet.h5",  # whenet snapshot path
    score=0.3, # yolo confidence score threshold
    iou=0.3, # yolo iou threshold
    gpu='0',
)

## Set env
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

time: 4.64 ms (started: 2021-07-17 14:21:22 +00:00)


In [17]:
## Init head pose estimation model
whenet = WHENet(snapshot=args.snapshot)

16809984/16804768 [==============================] - 2s 0us/step
time: 3.86 s (started: 2021-07-17 14:21:51 +00:00)


In [46]:
%%time
image_bgr = cv2.imread("/content/HeadPoseEstimation-WHENet/Sample/maruti.jpg")
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

faces, result_img = face_detector.detect_highlight_face(image_rgb, max_side_len=640)

logger.info(f"Total faces: {len(faces)}")
# logger.info(f"faces: {faces}")

bboxes = []
for face in faces:
    bbox = [faces[0]["x1"], faces[0]["y1"], faces[0]["x2"], faces[0]["y2"]]
    bboxes.append(bbox)

## Visualize
img_pil = Image.fromarray(image_rgb)
for bbox in bboxes:
    out_img, yaw, pitch, roll = process_detection(whenet, image_bgr, bbox, args)

logger.info(f"Storing result in out.jpg ... \n")
cv2.imwrite("out.jpg", out_img)

15:00:31 | INFO | __main__ | scale_factor: 0.16740779492545121
15:00:31 | INFO | __main__ | Total faces: 1
15:00:31 | INFO | __main__ | Storing result in out.jpg ... 

CPU times: user 588 ms, sys: 4.02 ms, total: 592 ms
Wall time: 527 ms
time: 536 ms (started: 2021-07-17 15:00:31 +00:00)
